In [1]:
from Dataset.Dataset import CSVLoader
from compoundFeaturization.rdkitFingerprints import MorganFingerprint
from featureSelection.baseFeatureSelector import LowVarianceFS
from splitters.splitters import SingletaskStratifiedSplitter
from models.sklearnModels import SklearnModel
from metrics.Metrics import Metric
from metrics.metricsFunctions import roc_auc_score, precision_score, accuracy_score, confusion_matrix, classification_report
from parameterOptimization.HyperparameterOpt import GridHyperparamOpt
from imbalanced_learn.ImbalancedLearn import SMOTEENN

from sklearn.ensemble import RandomForestClassifier

In [2]:
#Load Dataset
dataset = CSVLoader('preprocessed_dataset_wfoodb.csv', 'Smiles', ['Class'], 'ID')#, chunk_size=2000)
print(dataset.get_shape())

(23290,) (23290,) (0,) (23290,)
((23290,), (23290,), (0,), (23290,))


In [3]:
#Featurization
dataset = MorganFingerprint().featurize(dataset)
dataset.get_shape()

Featurizing datapoint 0
Featurizing datapoint 1000
Featurizing datapoint 2000
Featurizing datapoint 3000
Featurizing datapoint 4000
Featurizing datapoint 5000
Featurizing datapoint 6000


RDKit ERROR: [12:02:36] Explicit valence for atom # 1 Cl, 4, is greater than permitted


error in smile: O=[Cl]=O
Featurizing datapoint 7000


RDKit ERROR: [12:02:39] Explicit valence for atom # 3 B, 4, is greater than permitted
RDKit ERROR: [12:02:39] Explicit valence for atom # 1 Cl, 9, is greater than permitted


error in smile: OB1O[B]2(O)OB(O)O[B](O)(O1)O2
error in smile: O=[Cl-](=O)(=O)=O
Featurizing datapoint 8000
Featurizing datapoint 9000
Featurizing datapoint 10000
Featurizing datapoint 11000
Featurizing datapoint 12000
Featurizing datapoint 13000
Featurizing datapoint 14000


RDKit ERROR: [12:02:55] Explicit valence for atom # 0 P, 11, is greater than permitted


error in smile: [P](OCC=C(C)C)(OCC=C(C)C)(=O)(OP(OCC=C(C)C)(OCC=C(C)C)=O)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)CC=C(C)C
Featurizing datapoint 15000
Featurizing datapoint 16000
Featurizing datapoint 17000
Featurizing datapoint 18000
Featurizing datapoint 19000
Featurizing datapoint 20000
Featurizing datapoint 21000
Featurizing datapoint 22000
Featurizing datapoint 23000
Elements with indexes:  [6257, 7708, 7709, 14244]  were removed due to the presence of NAs!
The elements in question are:  ['O=[Cl]=O' 'OB1O[B]2(O)OB(O)O[B](O)(O1)O2' 'O=[Cl-](=O)(=O)=O'
 '[P](OCC=C(C)C)(OCC=C(C)C)(=O)(OP(OCC=C(C)C)(OCC=C(C)C)=O)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)CC=C(C)C']
(23286,) (23286,) (23286, 1024) (23286,)


((23286,), (23286,), (23286, 1024), (23286,))

In [4]:
#Feature Selection
dataset = LowVarianceFS(0.15).featureSelection(dataset)
dataset.get_shape()

(23286,) (23286,) (23286, 49) (23286,)


((23286,), (23286,), (23286, 49), (23286,))

In [5]:
#Data Split
splitter = SingletaskStratifiedSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset=dataset, frac_train=0.6, 
                                                                             frac_valid=0.2, frac_test=0.2)

In [6]:
train_dataset = SMOTEENN().sample(train_dataset)
train_dataset.get_shape()

((13968,), (22845,), (22845, 49), (13968,))

In [7]:
#Build a model function for hyperparameter optimization
def rf_model_builder(n_estimators, max_features, class_weight, model_dir=None):
    rf_model = RandomForestClassifier(n_estimators=n_estimators, max_features=max_features, 
                                      class_weight=class_weight)
    return SklearnModel(rf_model, model_dir)

params_dict_rf = {"n_estimators": [10, 100],
                  "max_features": ["auto", "sqrt", "log2", None],
                  "class_weight": [{0: 1., 1: 1.}, {0: 1., 1: 5}, {0: 1., 1: 10}]
                  }
    

In [8]:
#Hyperparameter Optimization
optimizer = GridHyperparamOpt(rf_model_builder)

best_rf, best_hyperparams, all_results = optimizer.hyperparam_search(params_dict_rf, train_dataset, 
                                                                     valid_dataset, Metric(roc_auc_score))

print('#################')
print(best_hyperparams)
print(best_rf)

Fitting 15 random models from a space of 24 possible models.
Fitting model 1/15
hyperparameters: {'n_estimators': 10, 'max_features': 'auto', 'class_weight': {0: 1.0, 1: 1.0}}
roc_auc_score: 
 0.7617569133962576
Model 1/15, Metric roc_auc_score, Validation set 1: 0.761757
	best_validation_score so far: 0.761757
Fitting model 2/15
hyperparameters: {'n_estimators': 10, 'max_features': 'auto', 'class_weight': {0: 1.0, 1: 5}}
roc_auc_score: 
 0.7491410001655904
Model 2/15, Metric roc_auc_score, Validation set 2: 0.749141
	best_validation_score so far: 0.761757
Fitting model 3/15
hyperparameters: {'n_estimators': 10, 'max_features': 'sqrt', 'class_weight': {0: 1.0, 1: 1.0}}
roc_auc_score: 
 0.7635370094386487
Model 3/15, Metric roc_auc_score, Validation set 3: 0.763537
	best_validation_score so far: 0.763537
Fitting model 4/15
hyperparameters: {'n_estimators': 10, 'max_features': 'sqrt', 'class_weight': {0: 1.0, 1: 5}}
roc_auc_score: 
 0.7396713031958935
Model 4/15, Metric roc_auc_score, Va

In [9]:
#Evaluate model
metrics = [Metric(roc_auc_score), Metric(precision_score), Metric(accuracy_score), Metric(confusion_matrix), 
           Metric(classification_report)]
best_rf.evaluate(test_dataset, metrics)

roc_auc_score: 
 0.7509841664726709
precision_score: 
 0.32608695652173914
accuracy_score: 
 0.9093642611683849
confusion_matrix: 
 [[4084  310]
 [ 112  150]]
classification_report: 
               precision    recall  f1-score   support

           0       0.97      0.93      0.95      4394
           1       0.33      0.57      0.42       262

    accuracy                           0.91      4656
   macro avg       0.65      0.75      0.68      4656
weighted avg       0.94      0.91      0.92      4656



{'roc_auc_score': 0.7509841664726709,
 'precision_score': 0.32608695652173914,
 'accuracy_score': 0.9093642611683849,
 'confusion_matrix': 1164.0,
 'classification_report': None}